In [ ]:
# Import Everything 

# Data storage / Manipuation 
import numpy as np
import pandas as pd

# Regression modelling 
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.model_selection import train_test_split

# Used for Acc metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import AdaBoostClassifier

from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [ ]:
df = pd.read_excel('/content/nfl_elo_Data.xlsx')
df.head()

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,...,qbelo_prob1,qbelo_prob2,qb1_game_value,qb2_game_value,qb1_value_post,qb2_value_post,qbelo1_post,qbelo2_post,score1,score2
0,2011-09-08,2011,0,NaN,GB,NO,1661.483,1562.051,0.720420,0.279580,...,0.682401,0.317599,323.863988,371.617931,218.154347,230.056137,1657.148002,1555.881727,42,34
1,2011-09-11,2011,0,NaN,WSH,NYG,1436.016,1538.007,0.446966,0.553034,...,0.386364,0.613636,172.685349,50.423742,93.326050,130.451209,1460.543814,1515.707558,28,14
2,2011-09-11,2011,0,NaN,SF,SEA,1464.780,1441.102,0.624917,0.375083,...,0.654592,0.345408,169.038179,20.054978,108.927018,85.817110,1495.356528,1408.892078,33,17
3,2011-09-11,2011,0,NaN,KC,BUF,1476.341,1439.839,0.642055,0.357945,...,0.675359,0.324641,29.181918,291.237536,85.025234,127.527651,1425.509540,1453.108743,7,41
4,2011-09-11,2011,0,NaN,CLE,CIN,1444.456,1449.716,0.585135,0.414865,...,0.611635,0.388365,71.536639,21.502224,62.074493,89.886992,1403.813926,1442.433574,17,27


In [ ]:
df['diff'] = df['score1'] - df['score2']
df['team1Result'] = df['diff'] > 0
df['team1Result'] = df['team1Result'].astype(int)
df.head()

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,...,qb1_game_value,qb2_game_value,qb1_value_post,qb2_value_post,qbelo1_post,qbelo2_post,score1,score2,diff,team1Result
0,2011-09-08,2011,0,NaN,GB,NO,1661.483,1562.051,0.720420,0.279580,...,323.863988,371.617931,218.154347,230.056137,1657.148002,1555.881727,42,34,8,1
1,2011-09-11,2011,0,NaN,WSH,NYG,1436.016,1538.007,0.446966,0.553034,...,172.685349,50.423742,93.326050,130.451209,1460.543814,1515.707558,28,14,14,1
2,2011-09-11,2011,0,NaN,SF,SEA,1464.780,1441.102,0.624917,0.375083,...,169.038179,20.054978,108.927018,85.817110,1495.356528,1408.892078,33,17,16,1
3,2011-09-11,2011,0,NaN,KC,BUF,1476.341,1439.839,0.642055,0.357945,...,29.181918,291.237536,85.025234,127.527651,1425.509540,1453.108743,7,41,-34,0
4,2011-09-11,2011,0,NaN,CLE,CIN,1444.456,1449.716,0.585135,0.414865,...,71.536639,21.502224,62.074493,89.886992,1403.813926,1442.433574,17,27,-10,0


In [ ]:
y = df['team1Result']
X = df[['elo1_pre','elo2_pre','elo_prob1','elo_prob2', 'qbelo1_pre','qbelo2_pre','qb1_value_pre','qb2_value_pre','qb1_adj','qb2_adj','qbelo_prob1',	'qbelo_prob2',	'qb1_game_value',	'qb2_game_value']]

In [ ]:
print(y.shape)
print(X.shape)

(2672,)
(2672, 14)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state= 42)


In [ ]:
dummy_classifier = DummyClassifier(strategy='most_frequent')
dummy_classifier.fit(X_train,y_train)
baseline_acc = dummy_classifier.score(X_test,y_test)

print("Baseline Accuracy = ", baseline_acc)

Baseline Accuracy =  0.5822942643391521


In [ ]:
# bagging 
model_bagging = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=10, random_state=42)
model_bagging.fit(X_train, y_train)
pred_bagging = model_bagging.predict(X_test)
acc_bagging = accuracy_score(y_test, pred_bagging)
print('Accuracy = ', acc_bagging)

Accuracy =  0.7356608478802993


In [ ]:
# Random Forest
model_rf = RandomForestClassifier(n_estimators=100, max_features=7, random_state=42)
model_rf.fit(X_train, y_train)
predict_rf = model_rf.predict(X_test)

print(accuracy_score(y_test, predict_rf))

0.7618453865336658


In [ ]:
# top 3 features for random forest model
feature_importances = model_rf.feature_importances_
features = X_train.columns
df = pd.DataFrame({'features': features, 'importance': feature_importances})
df = df.sort_values(by='importance', ascending=False)
print(df.iloc[0:10, :])

          features  importance
12  qb1_game_value    0.222490
13  qb2_game_value    0.184116
11     qbelo_prob2    0.085041
10     qbelo_prob1    0.069859
6    qb1_value_pre    0.051537
7    qb2_value_pre    0.049187
8          qb1_adj    0.048059
9          qb2_adj    0.045318
2        elo_prob1    0.043077
3        elo_prob2    0.042230


In [ ]:
# boosting 
base_est = DecisionTreeClassifier(max_depth=4)
ada_boost = AdaBoostClassifier(base_est, n_estimators=200, random_state=42, learning_rate=.05)
ada_boost.fit(X_train, y_train)
print(accuracy_score(y_test, ada_boost.predict(X_test)))

0.7381546134663342


In [ ]:
# voting
rfClf = RandomForestClassifier(n_estimators=200) # 500 trees. 
svmClf = SVC(probability=True) # probability calculation
logClf = LogisticRegression()
dtClf = DecisionTreeClassifier(max_depth=4)

# constructing the ensemble classifier by mentioning the individual classifiers.
clf2 = VotingClassifier(estimators = [('rf',rfClf), ('svm',svmClf), ('log', logClf), ('dtClf', dtClf)], voting='soft') 

# train the ensemble classifier
clf2.fit(X_train, y_train)
clf2_pred = clf2.predict(X_test)

print('Accuracy score', accuracy_score(y_test, clf2_pred))

Accuracy score 0.7817955112219451


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
